In [ ]:
pip install --upgrade numpy


In [ ]:
import numpy as np
import re as reg
from sympy import *
import math

import random


Variable reduction

In [ ]:
import sympy as sp

def reduce_objective(objective_str, constraints):
    """
    Reduces the objective function using the given constraints.

    Args:
        objective_str: The objective function as a string.
        constraints: A list of sympy equations representing the constraints.

    Returns:
        The reduced objective function as a sympy expression.
    """

    objective = sp.sympify(objective_str)
    variables = objective.free_symbols

    for constraint in constraints:
        left, right = constraint.args
        for var in left.free_symbols:
            if var in variables:
                # Check if the variable is involved in complex operations
                if var in objective.atoms(sp.Function):
                    continue

                # Try to solve the constraint for the variable
                try:
                    solved_var = sp.solve(constraint, var)[0]
                    objective = objective.subs(var, solved_var)
                    variables -= {var}
                except Exception as e:
                    print(f"Error solving for variable {var}: {e}")

    return objective

# Example usage
objective_str = ""
constraints = [
    # put the equations here
    sp.Eq(),

]

reduced_objective = reduce_objective(objective_str, constraints)



**DE**

In [ ]:

def x1_func(x2_val):
    return (abs(-x2_val**3 - 1))**(1/3)

def x4_func(x2_val, x5_val):
    return np.sqrt(((x2_val**2 - x1_func(x2_val)**2 - x5_val**2 + 10) * x2_val**2) / (25 * x5_val**2 + x2_val**2))

def x3_func(x2_val, x4_val, x5_val):
    return 5 * x4_val * x5_val / x2_val


# Define function to evaluate a solution (x1, x4, x3) for given x2 and x5 values
def evaluate_solution(x2_val, x5_val):
    # Calculate x1, x4, and x3 using defined functions
    x1_val = x1_func(x2_val)
    x4_val = x4_func(x2_val, x5_val)
    x3_val = x3_func(x2_val, x4_val, x5_val)

    # Calculate objective function (replace with your actual objective function)
    objective_function = reduced_objective

    # Consider returning additional values if needed
    return objective_function, x1_val, x4_val, x3_val

# DE algorithm parameters (adjust as needed)
population_size = 100
F = 0.8  # Scaling factor
CR = 0.3  # Crossover rate
generations = 200

# Define lower and upper bounds for x2 and x5
x2_lower_bound = -2.3
x2_upper_bound = 2.3
x5_lower_bound = -3.2
x5_upper_bound = 3.2

# Perform the optimization loop 30 times and store results
all_best_solutions = []
all_best_fitness_objective = []
run_index=0
fitness_evaluations=0
for run_index in range(30):
    # Initialize population (reset for each run)
    population = np.random.rand(population_size, 5) * (x2_upper_bound - x2_lower_bound) + x2_lower_bound
    population[:, 1:] = np.random.rand(population_size, 4) * (x5_upper_bound - x5_lower_bound) + x5_lower_bound

    # Function minimization loop
    best_solution = None
    best_fitness = float('inf')  # Initialize with positive infinity
    for generation in range(generations):
        for i in range(population_size):
            # Mutation
            mutant_vector = np.zeros_like(population[i])
            indices = np.random.choice(population_size, size=3, replace=False)
            mutant_vector += F * (population[indices[1], :] - population[indices[2], :])

            # Crossover
            trial_vector = population[i, :].copy()
            for j in range(len(trial_vector)):
                if random.random() < CR or j == 0:  # Ensure at least one element from mutant
                    trial_vector[j] = mutant_vector[j]

            # Bounds check and clipping
            trial_vector = np.clip(trial_vector, x2_lower_bound, x2_upper_bound)
            trial_vector[1:] = np.clip(trial_vector[1:], x5_lower_bound, x5_upper_bound)

            # Fitness evaluation
            trial_fitness, trial_x1, trial_x4, trial_x3 = evaluate_solution(trial_vector[0], trial_vector[1:])
            trial_fitness= np.average(trial_fitness)
            fitness_evaluations += 1
            # Selection (replace with your selection logic)
            if trial_fitness < best_fitness:
                best_solution = trial_vector.copy()
                best_fitness = trial_fitness

    all_best_fitness_objective = np.append(all_best_fitness_objective, best_fitness)




In [ ]:

print("Mean = " ,np.mean(all_best_fitness_objective),"\n")
print("Median = ",np.median(all_best_fitness_objective), "\n")
print("Worst = ",np.max(all_best_fitness_objective), "\n")
print("Best = ",np.min(all_best_fitness_objective), "\n")
print("Std = ", np.std(all_best_fitness_objective, ddof=1))
print(fitness_evaluations)
print(all_best_fitness_objective)

**PSO**

In [ ]:
import numpy as np
import random

def x1_func(x2_val):
    return (abs(-x2_val**3 - 1))**(1/3)

def x4_func(x2_val, x5_val):
    return np.sqrt(abs(((x2_val**2 - x1_func(x2_val)**2 - x5_val**2 + 10) * x2_val**2) / (25 * x5_val**2 + x2_val**2)))

def x3_func(x2_val, x4_val, x5_val):
    return 5 * x4_val * x5_val / x2_val


# Define function to evaluate a solution (x1, x4, x3) for given x2 and x5 values
def evaluate_solution(x2_val, x5_val):
    # Calculate x1, x4, and x3 using defined functions
    x1_val = x1_func(x2_val)
    x4_val = x4_func(x2_val, x5_val)
    x3_val = x3_func(x2_val, x4_val, x5_val)

    # Calculate objective function (replace with your actual objective function)
    objective_function = reduced_objective

    # Consider returning additional values if needed
    return objective_function, x1_val, x4_val, x3_val


# PSO algorithm parameters
population_size = 100
w = 0.9  # Inertia weight
c1 = 1.49445  # Cognitive coefficient
c2 = 1.49445  # Social coefficient
generations = 200

# Define lower and upper bounds for x2 and x5
x2_lower_bound = -2.3
x2_upper_bound = 2.3
x5_lower_bound = -3.2
x5_upper_bound = 3.2

# Perform the optimization loop 30 times and store results
all_best_solutions = []
all_best_fitness_objective = []
run_index = 0
fitness_evaluations = 0
for run_index in range(30):
  # Initialize population (reset for each run)
  population = np.random.rand(population_size, 5) * (x2_upper_bound - x2_lower_bound) + x2_lower_bound
  population[:, 1:] = np.random.rand(population_size, 4) * (x5_upper_bound - x5_lower_bound) + x5_lower_bound

  # Initialize particle velocities
  velocity = np.zeros_like(population)

  # Initialize personal best positions and fitness
  pbest_position = population.copy()
  pbest_fitness = np.ones(population_size) * float('inf')  # Initialize with positive infinity

  # Function minimization loop
  for generation in range(generations):
    # Evaluate fitness for each particle
    for i in range(population_size):
      fitness_evaluation, trial_x1, trial_x4, trial_x3 = evaluate_solution(population[i, 0], population[i, 1:])
      fitness_evaluations += 1
      fitness_evaluation_avg=np.average(fitness_evaluation)
      pbest_fitness[i] = min(pbest_fitness[i], fitness_evaluation_avg)  # Update personal best
    # Find global best position and fitness
    gBest_index = np.argmin(pbest_fitness)
    gBest_position = pbest_position[gBest_index]
    # Update velocity
    for i in range(population_size):
      # Cognitive component
      cognitive_term = c1 * random.random() * (pbest_position[i] - population[i])
      # Social component
      social_term = c2 * random.random() * (gBest_position - population[i])
      velocity[i] = w * velocity[i] + cognitive_term + social_term

    # Clip velocity to bounds (optional)
    velocity = np.clip(velocity, -abs(population - x2_lower_bound), abs(population - x2_upper_bound))
    velocity[:, 1:] = np.clip(velocity[:, 1:], -abs(population[:, 1:] - x5_lower_bound), abs(population[:, 1:] - x5_upper_bound))

    # Update position
    population = population + velocity

    # Clip position to bounds
    population = np.clip(population, x2_lower_bound, x2_upper_bound)
    population[:, 1:] = np.clip(population[:, 1:], x5_lower_bound, x5_upper_bound)

    # Find global best position and fitness
    # gBest_index = np.argmin(pbest_fitness)
    # gBest_position = pbest_position[gBest_index]

  all_best_fitness_objective.append(pbest_fitness[gBest_index])

print("All best fitness objectives:", all_best_fitness_objective)
print("Total fitness evaluations:", fitness_evaluations)


** L-Shade**

In [ ]:

# Define functions for x1, x4, and x3 (assuming x2 and x5 are numerical)
def x1_func(x2_val):
  """Calculates x1 based on x2."""
  # Replace with your actual function definition for x1 based on x2
  return (abs(-x2_val**3 - 1))**(1/3)  # Example calculation

def x4_func(x2_val, x5_val):
  """Calculates x4 based on x2 and x5."""
  # Replace with your actual function definition for x4 based on x2 and x5
  return np.sqrt(abs(((x2_val**2 - x1_func(x2_val)**2 - x5_val**2 + 10) * x2_val**2) / (25 * x5_val**2 + x2_val**2)))  # Example calculation

def x3_func(x2_val, x4_val, x5_val):
  """Calculates x3 based on x2, x4, and x5."""
  # Replace with your actual function definition for x3 based on x2, x4, and x5
  return 5 * x4_val * x5_val / x2_val  # Example calculation


# Define function to evaluate a solution (x1, x4, x3) for given x2 and x5 values
def evaluate_solution(x2_val, x5_val):
  """Calculates the objective function based on x1, x4, and x3."""
  # Calculate x1, x4, and x3 using defined functions
  x1_val = x1_func(x2_val)
  x4_val = x4_func(x2_val, x5_val)
  x3_val = x3_func(x2_val, x4_val, x5_val)

  # Calculate objective function (replace with your actual objective function)
  objective_function = np.exp(x1_val * x2_val * x3_val * x4_val * x5_val)

  # Consider returning additional values if needed
  return objective_function, x1_val, x4_val, x3_val


# L-SHADE algorithm parameters (adjust as needed)
population_size = 100
F = 0.5  # Initial scaling factor (can be adapted)
CR = 0.9  # Initial crossover rate (can be adapted)
generations = 200
archive_size = 20  # Size of the external archive

# Define lower and upper bounds for x2 and x5
x2_lower_bound = -2.3
x2_upper_bound = 2.3
x5_lower_bound = -3.2
x5_upper_bound = 3.2

# Perform the optimization loop 30 times and store results
all_best_solutions = []
all_best_fitness_objective = []
run_index = 0
fitness_evaluations = 0
for run_index in range(30):
  # Initialize population (reset for each run)
  population = np.random.rand(population_size, 5) * (x2_upper_bound - x2_lower_bound) + x2_lower_bound
  population[:, 1:] = np.random.rand(population_size, 4) * (x5_upper_bound - x5_lower_bound) + x5_lower_bound

  # Initialize external archive (empty for the first run)
  archive = [] if run_index == 0 else archive[:archive_size]

  # Function minimization loop
  best_solution = None
  best_fitness = float('inf')  # Initialize with positive infinity
  for generation in range(generations):
    # Differential Mutation with Strategy Adaptation
    for i in range(population_size):
      # Select random distinct indices (ensuring i is not included)
      indices = np.random.choice(population_size, size=5, replace=False)
      target_idx = np.random.randint(population_size)  # Randomly select a target individual

      # Randomly choose a strategy based on a probability distribution (e.g., rand < 0.5)
          # Choose a strategy based on probability distribution (replace with your implementation)
    if random.random() < 0.5:
      # DE/rand/1 strategy
      mutant_vector = population[indices[0], :] + F * (population[indices[1], :] - population[indices[2], :])
    else:
      # DE/current-to-rand/1 strategy (replace with other strategies if needed)
      mutant_vector = population[target_idx, :] + F * (population[indices[1], :] - population[indices[2], :])

    # Crossover
    trial_vector = population[i, :].copy()
    for j in range(len(trial_vector)):
      if random.random() < CR or j == 0:  # Ensure at least one element from mutant
        trial_vector[j] = mutant_vector[j]

    # Bounds check and clipping
    trial_vector = np.clip(trial_vector, x2_lower_bound, x2_upper_bound)
    trial_vector[1:] = np.clip(trial_vector[1:], x5_lower_bound, x5_upper_bound)

    # Fitness evaluation
    trial_fitness, trial_x1, trial_x4, trial_x3 = evaluate_solution(trial_vector[0], trial_vector[1:])
    fitness_evaluations += 1

    # Selection with Archive Integration
    if trial_fitness.all() < best_fitness:
      best_solution = trial_vector.copy()
      best_fitness = np.average(trial_fitness)

    # Update archive (consider adding or replacing based on fitness)

    if len(archive) < archive_size:
      # archive.append((trial_fitness, trial_vector.copy()))
      archive.append(np.average(trial_fitness))
    else:
      # Implement your archive update strategy (e.g., replace worst element)
        if archive:
          # worst_index = np.argmax(np.array([item[0] for item in archive]))
          worst_index = np.argmax(archive)
          if np.average(trial_fitness) < archive[worst_index]:
            archive[worst_index] = np.average(trial_fitness)

  F_update_threshold = 0.6  # Threshold for successful updates in archive
  CR_update_threshold = 0.8  # Threshold for successful updates in archive
  F_lower_bound = 0.2  # Lower bound for F
  F_upper_bound = 0.8  # Upper bound for F
  CR_lower_bound = 0.1  # Lower bound for CR
  CR_upper_bound = 0.9  # Upper bound for CR

# Calculate success rate in archive
  success_rate = np.mean(archive)

# Update F
  if success_rate > F_update_threshold:
    F = min(F * 1.1, F_upper_bound)  # Increase F slightly if successful updates are high
  else:
    F = max(F * 0.9, F_lower_bound)  # Decrease F slightly if successful updates are low

# Update CR
  if success_rate > CR_update_threshold:
    CR = min(CR * 1.2, CR_upper_bound)  # Increase CR slightly if successful updates are high
  else:
    CR = max(CR * 0.8, CR_lower_bound)
    # ... (code to update F and CR based on archive data)
  all_best_fitness_objective.append(best_fitness)
    # Print progress or store results (optional)
    # ... (code to print or store results for each generation)

  # Print final results or perform post-processing (optional)
print("All best fitness:", all_best_fitness_objective)



All best fitness: [582.8375018531456, 3.3187746937507363, 18.22232607371885, 16.409228679511248, 110.52567986757278, 21364864.326866098, 1.0477957089240144, 40988946.33438608, 15122181.422503375, 59.07691553744456, 1.0002826458769538, 234.76794700968657, 6.018942302425537, 31251.904714995762, 4122330.283815614, 34564.52461153019, 1.4791829439855662, 4.319451652474834, 444.1818528982003, 1.008063866947704, 37.072153702946984, 316.74897945714486, 41622019.927806914, 1.0561296868850936, 653.0781461312696, 3.8794072147707928, 2.470153438641253, 204.10948624346653, 17962118.697940763, 4.536160530898161]


In [ ]:

print(F,CR)
print(archive)
print( trial_x1, trial_x4, trial_x3)

0.8 0.9
[1.000065062186251, 1.00002706725875, 1.0000013638223229, 1.0000533493686472, 1.0000094125907304, 1.0000662316742366, 1.0000599733082804, 1.0000110261988238, 1.0000342156448225, 1.000009755459147, 1.0000568220788801, 1.0000298265088725, 1.0000124435086737, 1.0000356590508985, 1.0000568569661394, 1.0000219249954878, 1.0000021470366713, 1.0000193753975353, 1.0000718967901092, 1.0000081303644552]
1.45289626456921 [0.22620754 0.22620754 0.22620754 1.9164027 ] [-2.04219092 -2.04219092  2.04219092  2.39535243]


Test

In [ ]:
fitness_evaluation, trial_x1, trial_x4, trial_x3 = evaluate_solution(population[0, 0], population[0, 1:])
print(fitness_evaluation, trial_x1, trial_x4, trial_x3)
pbest_fitness[i] = min(pbest_fitness[i], fitness_evaluation.any())

[2.73074106 3.27444134 2.73074106 4.67907896] 0.7440462339262577 [0.22593211 0.27238586 0.22593211 0.42615397] [ 2.31600971 -2.51662743  2.31600971 -2.87042918]


ACO

In [ ]:
import numpy as np
import random

def evaluate_solution(x2_val, x5_val):
    # ... your original function

def aco(num_ants, max_iter, alpha, beta, rho):
    # Problem dimensions
    dim = 2  # Number of variables (x2 and x5)

    # Bounds for variables
    lower_bound = np.array([-2.3, -3.2])
    upper_bound = np.array([2.3, 3.2])

    # Initialize pheromone matrix
    pheromone_matrix = np.ones((population_size, population_size))

    # Best solution and its fitness
    best_solution = None
    best_fitness = float('inf')

    for iter in range(max_iter):
        # Create ants
        ants = np.random.uniform(lower_bound, upper_bound, size=(num_ants, dim))

        # Construct solutions and evaluate fitness
        for ant_idx in range(num_ants):
            fitness, _, _, _ = evaluate_solution(ants[ant_idx, 0], ants[ant_idx, 1])
            # Update best solution if necessary
            if fitness < best_fitness:
                best_solution = ants[ant_idx].copy()
                best_fitness = fitness

            # Update pheromone matrix (implement your pheromone update strategy)

        # Pheromone evaporation
        pheromone_matrix *= (1 - rho)

    return best_solution, best_fitness

# Example usage
num_ants = 50
max_iter = 100
alpha = 1.0  # Pheromone influence
beta = 2.0  # Heuristic information influence
rho = 0.1  # Evaporation rate

best_solution, best_fitness = aco(num_ants, max_iter, alpha, beta, rho)



Main Code